In [3]:
pip install faker mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import mysql.connector
from faker import Faker
import hashlib
from datetime import datetime, timedelta
import random

# Populate UserInfo table

In [2]:
# Initialize Faker
fake = Faker()

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456789!",
    database="User_DB"
)
cursor = conn.cursor()

# Function to generate hashed passwords
def hash_password(password):
    return hashlib.md5(password.encode()).hexdigest()  # MD5 for simplicity

# Generate and insert 100 fake users
for _ in range(100):
    first_name = fake.first_name()
    last_name = fake.last_name()
    age = random.randint(18, 60)
    gender = random.choice(['M', 'F'])
    dob = fake.date_of_birth(minimum_age=age, maximum_age=age)
    email = fake.unique.email()
    password_hash = hash_password(fake.password())

    sql = """INSERT INTO UserInfo (first_name, last_name, age, gender, dob, email, password_hash)
             VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    values = (first_name, last_name, age, gender, dob, email, password_hash)
    
    cursor.execute(sql, values)

# Commit and close connection
conn.commit()
cursor.close()
conn.close()

print("100 synthetic users inserted successfully!")


100 synthetic users inserted successfully!


# Populate HealthMetrics table

In [6]:
# Initialize Faker
fake = Faker()

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456789!",
    database="User_DB"
)
cursor = conn.cursor()

# Fetch all user IDs from UserInfo
cursor.execute("SELECT user_id FROM UserInfo")
user_ids = [row[0] for row in cursor.fetchall()]

# Function to generate synthetic health data
def generate_health_data():
    avg_exercise_per_week = random.randint(0, 10)  # 0-10 hours per week
    avg_sleep_hours = random.randint(4, 10)  # 4-10 hours per day
    stress_level = random.randint(1, 10)  # 1 (low stress) to 10 (high stress)
    
    diet_quality = random.choices(
        ['Poor', 'Average', 'Good', 'Very good'], 
        weights=[0.2, 0.4, 0.3, 0.1]  # More people have average diets
    )[0]
    
    substance_choices = ['Alcohol', 'Caffeine', 'Tobacco', 'None']
    most_freq_substance_used = random.choices(substance_choices, weights=[0.3, 0.4, 0.2, 0.1])[0]
    
    # Generate a random date within the last 365 days
    date_recorded = fake.date_between(start_date='-365d', end_date='today')
    
    return (avg_exercise_per_week, avg_sleep_hours, stress_level, diet_quality, most_freq_substance_used, date_recorded)

# Insert synthetic data for each user
for user_id in user_ids:
    health_data = generate_health_data()
    
    sql = """INSERT INTO HealthMetrics (user_id, date_recorded, avg_exercise_per_week, avg_sleep_hours, stress_level, diet_quality, most_freq_substance_used)
             VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    
    values = (user_id, health_data[5], health_data[0], health_data[1], health_data[2], health_data[3], health_data[4])
    
    cursor.execute(sql, values)

# Commit and close connection
conn.commit()
cursor.close()
conn.close()

print(f"{len(user_ids)} synthetic health records inserted successfully!")

303 synthetic health records inserted successfully!


# Populate Conditions, Medications, and ConditionsMedications tables

In [4]:
# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456789!",
    database="User_DB"
)
cursor = conn.cursor()

# Expanded conditions related to poor lifestyle
conditions = [
    ("Obesity",),
    ("Type 2 Diabetes",),
    ("Hypertension",),
    ("High Cholesterol",),
    ("Heart Disease",),
    ("Liver Disease (Fatty Liver)",),
    ("Chronic Obstructive Pulmonary Disease (COPD)",),
    ("Alcoholic Liver Disease",),
    ("Stroke",),
    ("Gastroesophageal Reflux Disease (GERD)",),
    ("Gout",),
    ("Osteoarthritis",),
    ("Depression & Anxiety",),
    ("Sleep Apnea",),
    ("Non-Alcoholic Fatty Liver Disease (NAFLD)",),
    ("Erectile Dysfunction",),
    ("Polycystic Ovary Syndrome (PCOS)",),
    ("Chronic Kidney Disease (CKD)",),
    ("Lung Cancer",),
    ("Colon Cancer",),
    ("Pancreatitis",),
    ("Cirrhosis",),
]

# Insert conditions into Conditions table
cursor.executemany("INSERT INTO Conditions (condition_name) VALUES (%s)", conditions)
conn.commit()

# Retrieve condition IDs
cursor.execute("SELECT condition_id, condition_name FROM Conditions")
condition_dict = {name: cid for cid, name in cursor.fetchall()}

# Define medications for each condition
medications = {
    "Obesity": ["Orlistat", "Phentermine", "Liraglutide", "Semaglutide"],
    "Type 2 Diabetes": ["Metformin", "Insulin", "Glipizide", "Dapagliflozin", "Sitagliptin"],
    "Hypertension": ["Lisinopril", "Amlodipine", "Hydrochlorothiazide", "Losartan", "Metoprolol"],
    "High Cholesterol": ["Atorvastatin", "Simvastatin", "Rosuvastatin", "Ezetimibe", "Fenofibrate"],
    "Heart Disease": ["Aspirin", "Clopidogrel", "Beta Blockers", "ACE Inhibitors", "Nitroglycerin"],
    "Liver Disease (Fatty Liver)": ["Vitamin E", "Pioglitazone", "Ursodeoxycholic Acid"],
    "Chronic Obstructive Pulmonary Disease (COPD)": ["Albuterol", "Tiotropium", "Ipratropium", "Prednisone"],
    "Alcoholic Liver Disease": ["Baclofen", "Pentoxifylline"],
    "Stroke": ["Alteplase", "Warfarin", "Aspirin", "Clopidogrel"],
    "Gastroesophageal Reflux Disease (GERD)": ["Omeprazole", "Esomeprazole", "Ranitidine", "Famotidine"],
    "Gout": ["Allopurinol", "Febuxostat", "Colchicine", "Probenecid"],
    "Osteoarthritis": ["Acetaminophen", "Ibuprofen", "Celecoxib", "Naproxen"],
    "Depression & Anxiety": ["Fluoxetine", "Sertraline", "Escitalopram", "Venlafaxine", "Bupropion"],
    "Sleep Apnea": ["Modafinil"],
    "Non-Alcoholic Fatty Liver Disease (NAFLD)": ["Pioglitazone", "Vitamin E", "Omega-3 Fatty Acids"],
    "Erectile Dysfunction": ["Sildenafil", "Tadalafil", "Vardenafil"],
    "Polycystic Ovary Syndrome (PCOS)": ["Metformin", "Spironolactone", "Oral Contraceptives"],
    "Chronic Kidney Disease (CKD)": ["Lisinopril", "Losartan", "Sodium Bicarbonate"],
    "Lung Cancer": ["Cisplatin", "Carboplatin", "Osimertinib", "Erlotinib"],
    "Colon Cancer": ["Fluorouracil", "Oxaliplatin"],
    "Pancreatitis": ["Pancreatic Enzyme Replacement Therapy", "Pain Relievers"],
    "Cirrhosis": ["Furosemide", "Spironolactone", "Lactulose"],
}

# Insert medications into Medications table
medication_names = set(med for med_list in medications.values() for med in med_list)
cursor.executemany("INSERT INTO Medications (medication_name) VALUES (%s)", [(med,) for med in medication_names])
conn.commit()

# Retrieve medication IDs
cursor.execute("SELECT medication_id, medication_name FROM Medications")
medication_dict = {name: mid for mid, name in cursor.fetchall()}

# Insert data into ConditionsMedications table
conditions_medications = [(condition_dict[c], medication_dict[m]) for c in medications for m in medications[c]]

cursor.executemany("INSERT INTO ConditionsMedications (condition_id, medication_id) VALUES (%s, %s)", conditions_medications)
conn.commit()

# Close connection
cursor.close()
conn.close()

print("Expanded conditions, medications, and mappings inserted successfully!")


Expanded conditions, medications, and mappings inserted successfully!


# Populate UserConditions and UserMedications tables

In [8]:
# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456789!",
    database="User_DB"
)
cursor = conn.cursor()

# Step 1: Retrieve all users
cursor.execute("SELECT user_id FROM UserInfo")
users = [row[0] for row in cursor.fetchall()]

# Step 2: Retrieve all conditions
cursor.execute("SELECT condition_id FROM Conditions")
conditions = [row[0] for row in cursor.fetchall()]

# Step 3: Retrieve all condition-medication mappings
cursor.execute("SELECT condition_id, medication_id FROM ConditionsMedications")
condition_medications = {}
for condition_id, medication_id in cursor.fetchall():
    if condition_id not in condition_medications:
        condition_medications[condition_id] = []
    condition_medications[condition_id].append(medication_id)

# Step 4: Assign conditions & medications to users
user_conditions = []
user_medications = []

for user_id in users:
    # Assign 1 to 3 random conditions per user
    assigned_conditions = random.sample(conditions, k=random.randint(1, 3))
    
    for condition_id in assigned_conditions:
        user_conditions.append((user_id, condition_id))
        
        # Assign medications linked to this condition
        if condition_id in condition_medications:
            for medication_id in condition_medications[condition_id]:
                user_medications.append((user_id, medication_id))

# Step 5: Insert assigned conditions into UserConditions
cursor.executemany("INSERT INTO UserConditions (user_id, condition_id) VALUES (%s, %s)", user_conditions)
conn.commit()

# Step 6: Insert assigned medications into UserMedications (avoid duplicates)
user_medications = list(set(user_medications))  # Remove duplicate entries
cursor.executemany("INSERT INTO UserMedications (user_id, medication_id) VALUES (%s, %s)", user_medications)
conn.commit()

# Close connection
cursor.close()
conn.close()

print("Conditions and medications assigned to users successfully!")


Conditions and medications assigned to users successfully!
